# Download essential datasets and packages

In [1]:
!wget https://www.dropbox.com/s/c1pzdacnzhvi6pm/histos_tba.20.npz

--2020-03-29 10:58:15--  https://www.dropbox.com/s/c1pzdacnzhvi6pm/histos_tba.20.npz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/c1pzdacnzhvi6pm/histos_tba.20.npz [following]
--2020-03-29 10:58:15--  https://www.dropbox.com/s/raw/c1pzdacnzhvi6pm/histos_tba.20.npz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc046f2f056b63425307734023ee.dl.dropboxusercontent.com/cd/0/inline/A00pXiCDDZvpZG2FgQYih9iSVflbBCaWilXy8SukOSczXbT2r8Ovtmksq7ST1LYhMgntGyXHMdoDSvo1m1tN7VQxkkxVZrCNi7WnXMgT3biRYwnAam1xSmq7jlsYkUnemHQ/file# [following]
--2020-03-29 10:58:15--  https://uc046f2f056b63425307734023ee.dl.dropboxusercontent.com/cd/0/inline/A00pXiCDDZvpZG2FgQYih9iSVflbBCaWilXy8SukOSczXbT2r8Ovtmksq7ST1LYhMgntGyXHMdoDSvo1m1tN7VQxkkxVZrCNi7W

Python module dependencies

In [0]:
# %tensorflow_version 1.x

In [1]:
import numpy as np
from scipy import stats
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
import re 
from sklearn.preprocessing import OneHotEncoder

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# Load data

In [0]:
data = np.load('histos_tba.20.npz')
variables = data['variables']
parameters = data['parameters']

# Data Preparation

In [0]:
variables = variables[:,:84]

parameter array 0th index contains charge/momentum, to get the momentum we have to get the negative reciprocal of the charge / momentum

In [4]:
transverse_momentum = -1 / np.delete(parameters, [1,2], 1) 
stats.describe(transverse_momentum)

DescribeResult(nobs=3272341, minmax=(array([-6989.3447], dtype=float32), array([6989.252], dtype=float32)), mean=array([0.03644617], dtype=float32), variance=array([16305.736], dtype=float32), skewness=array([-0.50260574], dtype=float32), kurtosis=array([991.76306], dtype=float32))

In [0]:
dataset = np.hstack((variables,transverse_momentum))

In [6]:
cleaned_dataset = []
for e in dataset:
  if e[84] > 0.0 and e[84] <= 10.0:
    e[84] = 1
    cleaned_dataset.append(e)
  elif e[84] > 10.0 and e[84] <= 30.0:
    e[84] = 2
    cleaned_dataset.append(e)
  elif e[84] > 30.0 and e[84] <= 100.0:
    e[84] = 3
    cleaned_dataset.append(e)
  elif e[84] > 100.0:
    e[84] = 4
    cleaned_dataset.append(e)
  else:
    e[84] = 5
    cleaned_dataset.append(e)

cleaned_dataset = np.asarray(cleaned_dataset)
cleaned_dataset

array([[3.190e+03,       nan, 3.546e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [2.865e+03,       nan, 2.622e+03, ..., 0.000e+00, 0.000e+00,
        5.000e+00],
       [      nan, 1.415e+03, 1.455e+03, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       ...,
       [4.054e+03,       nan, 4.016e+03, ..., 0.000e+00, 0.000e+00,
        5.000e+00],
       [      nan, 3.464e+03, 3.439e+03, ..., 1.000e+00, 1.000e+00,
        4.000e+00],
       [8.360e+02,       nan, 2.960e+02, ..., 0.000e+00, 1.000e+00,
        5.000e+00]], dtype=float32)

In [10]:
stats.describe(cleaned_dataset.T[84])

DescribeResult(nobs=3272341, minmax=(1.0, 5.0), mean=3.1621292, variance=3.6044843, skewness=-0.13364042341709137, kurtosis=-1.900529585828819)

In [0]:
X = cleaned_dataset[:,0:84]
X = np.nan_to_num(X)
y = cleaned_dataset[:,84]
y = y.astype(int)

# Downsampling data and creating images

Images are made in the shape of 12 X 7

this conforms to the detector planes and the number of features

In [8]:
print('Original dataset shape %s' % Counter(y))

rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))
X_res = X_res.reshape(185350,7,12)
y_res = y_res.reshape(-1)

Original dataset shape Counter({5: 1635011, 1: 1263726, 2: 249098, 3: 87436, 4: 37070})
Resampled dataset shape Counter({1: 37070, 2: 37070, 3: 37070, 4: 37070, 5: 37070})


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


# Train Test Split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42)

In [23]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils import data

# Check Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define Hyper-parameters 
input_size = 84
hidden_size = 500
hidden_size_l2 = 300
num_classes = 6
num_epochs = 30
batch_size = 100
learning_rate = 0.001

tensor_x = torch.Tensor(X_train) # transform to train torch tensor
tensor_y = torch.Tensor(y_train)
tensor_y = torch.tensor(tensor_y,dtype=torch.long, device=device)

test_tensor_x = torch.Tensor(X_test) # transform to test torch tensor
test_tensor_y = torch.Tensor(y_test)
test_tensor_y = torch.tensor(test_tensor_y,dtype=torch.long, device=device)

train_dataset = data.TensorDataset(tensor_x,tensor_y)
train_loader = data.DataLoader(train_dataset,batch_size=batch_size)

test_dataset = data.TensorDataset(test_tensor_x,test_tensor_y)
test_loader = data.DataLoader(test_dataset,batch_size=batch_size)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


# Fully Connected Neural Network

The parameters were made on looking at the euclidean data spread of the dataset.

In [0]:
# Fully connected neural network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size_l2)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size_l2, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [25]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 7*12).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backprpagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In the test phase, don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 7*12).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/30], Step [100/1298], Loss: 4.0190
Epoch [1/30], Step [200/1298], Loss: 3.1071
Epoch [1/30], Step [300/1298], Loss: 2.2777
Epoch [1/30], Step [400/1298], Loss: 1.7598
Epoch [1/30], Step [500/1298], Loss: 2.3973
Epoch [1/30], Step [600/1298], Loss: 1.3483
Epoch [1/30], Step [700/1298], Loss: 1.2184
Epoch [1/30], Step [800/1298], Loss: 1.1770
Epoch [1/30], Step [900/1298], Loss: 1.1317
Epoch [1/30], Step [1000/1298], Loss: 1.0188
Epoch [1/30], Step [1100/1298], Loss: 1.1775
Epoch [1/30], Step [1200/1298], Loss: 0.8917
Epoch [2/30], Step [100/1298], Loss: 1.0059
Epoch [2/30], Step [200/1298], Loss: 1.1653
Epoch [2/30], Step [300/1298], Loss: 0.9331
Epoch [2/30], Step [400/1298], Loss: 1.0214
Epoch [2/30], Step [500/1298], Loss: 0.9558
Epoch [2/30], Step [600/1298], Loss: 0.9886
Epoch [2/30], Step [700/1298], Loss: 0.9772
Epoch [2/30], Step [800/1298], Loss: 1.2125
Epoch [2/30], Step [900/1298], Loss: 0.7978
Epoch [2/30], Step [1000/1298], Loss: 0.9634
Epoch [2/30], Step [1100/129

# 2D Convolutional Neural Network 

epoch to epoch the CNN gives better accuracy.

The parameters of the models were defined by theoretical knowledge on CNNs such as strides and kernel size. I made them small since the the image size were training is small as well. ( 12 X 7)

In [0]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(384, 500)
        self.fc2 = nn.Linear(500, 6)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [0]:
modelConv = ConvNet()
modelConv.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelConv.parameters(), lr=learning_rate)

In [17]:
num_epochs = 10
# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        if len(images) != 100:
          continue
        labels = labels.to(device)
        # Run the forward pass
        outputs = modelConv(images.reshape(100,1,7,12).to(device))
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/10], Step [100/1298], Loss: 1.1642, Accuracy: 52.00%
Epoch [1/10], Step [200/1298], Loss: 0.9481, Accuracy: 60.00%
Epoch [1/10], Step [300/1298], Loss: 1.0917, Accuracy: 53.00%
Epoch [1/10], Step [400/1298], Loss: 1.0306, Accuracy: 56.00%
Epoch [1/10], Step [500/1298], Loss: 1.0861, Accuracy: 52.00%
Epoch [1/10], Step [600/1298], Loss: 1.0128, Accuracy: 55.00%
Epoch [1/10], Step [700/1298], Loss: 1.1650, Accuracy: 44.00%
Epoch [1/10], Step [800/1298], Loss: 1.0874, Accuracy: 57.00%
Epoch [1/10], Step [900/1298], Loss: 0.8103, Accuracy: 69.00%
Epoch [1/10], Step [1000/1298], Loss: 0.9047, Accuracy: 58.00%
Epoch [1/10], Step [1100/1298], Loss: 1.0228, Accuracy: 57.00%
Epoch [1/10], Step [1200/1298], Loss: 1.0219, Accuracy: 57.00%
Epoch [2/10], Step [100/1298], Loss: 1.0456, Accuracy: 52.00%
Epoch [2/10], Step [200/1298], Loss: 0.9450, Accuracy: 61.00%
Epoch [2/10], Step [300/1298], Loss: 1.1386, Accuracy: 50.00%
Epoch [2/10], Step [400/1298], Loss: 0.9722, Accuracy: 60.00%
Epoch

In [20]:
# Test the model
modelConv.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        if len(images) != 100:
          continue
        labels = labels.to(device)
        outputs = modelConv(images.reshape(100,1,7,12).to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on test images: {} %'.format((correct / total) * 100))

# Save the model and plot
torch.save(modelConv.state_dict(), '/' + 'conv_net_model.ckpt')

Test Accuracy of the model on the 10000 test images: 63.13669064748202 %
